<a href="https://colab.research.google.com/github/p-stehlik/StudentNotebooks/blob/main/PBSTimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

#load the data

In [ ]:
#LOAD THE PBS DATA

#To get the file path go to the file explorer tab, press the "three dots" and click "copy path"
#paste the file path into the quotation marks below - NOTE: the quotation marks tell python that its a string (sentence) and not a command
file_path = "/content/PBS_Data/dos-jul-2021-to-nov-2025.xlsx"

#the code below reads and .xlsx file, looks at all the sheets, and then merges them into one huge sheet
#dont change the code below - keep as is.

sheets = pd.read_excel(file_path, sheet_name=None)


df_PBS = pd.concat(
    [d.assign(sheet_name=name) for name, d in sheets.items()],
    ignore_index=True
)

#this creates a "data frame" which we have named "df_PBS"
#honestly you can name the dataframe bananaMOOOMOO for all the program cares, but convention is to put df_NAME, and for your readability give it a sensible name, and you cannot use spaces!
#so if you are for example looking at MBS data, you can change to df_MBS or something

In [ ]:
#have a look at the first few rows of df_PBS

df_PBS.head() # you can change the number of rows by putting a number in the

,MONTH_OF_SUPPLY,ITEM_CODE,ATC5_CODE,DRUG_NAME,PTNT_CTGRY_DRVD_CD,DRG_TYP_CTGRY,SCRIPT_TYPE,PRSCRPTN_CNT,PATIENT_CONTRIB,GOVT_CONTRIB,RETAIL_MARKUP,TOTAL_COST,PATIENT_NET_CONTRIB,sheet_name
0,202107,00000B,Z,MISSING ITEM CODE,R0,Unknown,ABOVE CO-PAYMENT,771,0.0,30747.73,0.0,30747.73,18.80,DOS_FY2021_22
1,202107,00000B,Z,MISSING ITEM CODE,R0,Unknown,UNDER CO-PAYMENT,25,6.6,0.00,0.0,6.60,89.72,DOS_FY2021_22
2,202107,00000B,Z,MISSING ITEM CODE,R1,Unknown,ABOVE CO-PAYMENT,795,5233.8,41013.11,0.0,46246.91,2868.49,DOS_FY2021_22
3,202107,00000B,Z,MISSING ITEM CODE,R1,Unknown,UNDER CO-PAYMENT,2,12.3,0.00,0.0,12.30,12.30,DOS_FY2021_22
4,202107,00013Q,Z,EXTEMPORANEOUSLY PREPARED,C0,Section 85,ABOVE CO-PAYMENT,1959,0.0,83309.92,0.0,83309.92,23.09,DOS_FY2021_22


In [10]:
# dataFrame.info() function lets you explore your dataframe
#it tells you the column names, the number of columns, the number of rows (entries) and the data types
#objects = categories
#int64 = a type of number

df_PBS.info()

#notice that the MONTH_OF_SUPPLY column is an int, not a date. We will need to convert this to a date format for a time series analysis!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210410 entries, 0 to 1210409
Data columns (total 14 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   MONTH_OF_SUPPLY      1210410 non-null  int64  
 1   ITEM_CODE            1210410 non-null  object 
 2   ATC5_CODE            1210410 non-null  object 
 3   DRUG_NAME            1210410 non-null  object 
 4   PTNT_CTGRY_DRVD_CD   1210410 non-null  object 
 5   DRG_TYP_CTGRY        1210410 non-null  object 
 6   SCRIPT_TYPE          1210410 non-null  object 
 7   PRSCRPTN_CNT         1210410 non-null  int64  
 8   PATIENT_CONTRIB      1210410 non-null  float64
 9   GOVT_CONTRIB         1210410 non-null  float64
 10  RETAIL_MARKUP        1210410 non-null  float64
 11  TOTAL_COST           1210410 non-null  float64
 12  PATIENT_NET_CONTRIB  1210410 non-null  float64
 13  sheet_name           1210410 non-null  object 
dtypes: float64(5), int64(2), object(7)
memory usage: 1

You should be able to see above that there are LOTS of rows, but you can see what is in each row.
It is important to remember that this now has ALL of the data but you will only need to look at whatever your

In [6]:
#create a list of item codes you want to search for
#each item code will need to be in quotation marks
#again you can call the list below whatever you want, but I thought PBS items might be intuative here, just give it a sensible name.

PBSItems = ["02666H",
            "02922T",
            "02922T"
            ]

 #now filter your large df_PBS and only get the rows of interest
 #the code below looks into the df_PBS dataframe, and then looks through the column called "ITEM_CODE" and finds which rows match any of the item codes you have listed in the "PBSItems" list

df_PBSFiltered = df_PBS[df_PBS['ITEM_CODE'].isin(PBSItems)]

df_PBSFiltered.head()

,MONTH_OF_SUPPLY,ITEM_CODE,ATC5_CODE,DRUG_NAME,PTNT_CTGRY_DRVD_CD,DRG_TYP_CTGRY,SCRIPT_TYPE,PRSCRPTN_CNT,PATIENT_CONTRIB,GOVT_CONTRIB,RETAIL_MARKUP,TOTAL_COST,PATIENT_NET_CONTRIB,sheet_name
3857,202107,02666H,J01EA01,TRIMETHOPRIM,C0,Section 85,ABOVE CO-PAYMENT,867,0.00,13499.56,3787.36,13499.56,79.58,DOS_FY2021_22
3858,202107,02666H,J01EA01,TRIMETHOPRIM,C1,Section 85,ABOVE CO-PAYMENT,1490,9682.20,13605.92,6555.12,23288.12,9418.82,DOS_FY2021_22
3859,202107,02666H,J01EA01,TRIMETHOPRIM,C1,Section 85,UNDER CO-PAYMENT,11,35.46,0.00,0.00,35.46,72.60,DOS_FY2021_22
3860,202107,02666H,J01EA01,TRIMETHOPRIM,G1,Section 85,ABOVE CO-PAYMENT,13,79.20,122.04,55.90,201.24,78.20,DOS_FY2021_22
3861,202107,02666H,J01EA01,TRIMETHOPRIM,G2,Section 85,ABOVE CO-PAYMENT,4,26.40,40.72,17.20,67.12,25.40,DOS_FY2021_22


In [8]:
#lets have a look at the new df structure
df_PBSFiltered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 879 entries, 3857 to 1186035
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   MONTH_OF_SUPPLY      879 non-null    int64  
 1   ITEM_CODE            879 non-null    object 
 2   ATC5_CODE            879 non-null    object 
 3   DRUG_NAME            879 non-null    object 
 4   PTNT_CTGRY_DRVD_CD   879 non-null    object 
 5   DRG_TYP_CTGRY        879 non-null    object 
 6   SCRIPT_TYPE          879 non-null    object 
 7   PRSCRPTN_CNT         879 non-null    int64  
 8   PATIENT_CONTRIB      879 non-null    float64
 9   GOVT_CONTRIB         879 non-null    float64
 10  RETAIL_MARKUP        879 non-null    float64
 11  TOTAL_COST           879 non-null    float64
 12  PATIENT_NET_CONTRIB  879 non-null    float64
 13  sheet_name           879 non-null    object 
dtypes: float64(5), int64(2), object(7)
memory usage: 103.0+ KB


In [9]:
len(df_PBSFiltered)

879